In [1]:
import pandas as pd
import os
import sys

In [2]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw

In [3]:
BW_PROJECT = 'metallican' # insert your project name here
bd.projects.set_current(BW_PROJECT)
bd.databases

Databases dictionary with 11 object(s):
	Copper
	Gold
	Iron
	Nickel
	Regioinvent
	Silver
	Zinc
	biosphere3
	biosphere3_spatialized_flows
	ecoinvent-3.10-cutoff
	ecoinvent-3.10-cutoff regionalized

# Regioinvent

In [8]:
# change the path here to wherever you stored the Regioinvent Python package
sys.path.append(r"C:\Users\mp_ma\OneDrive - polymtl\POST_DOC\Modules\Regioinvent\src")
import regioinvent

In [11]:
regio = regioinvent.Regioinvent(bw_project_name='metallican', ecoinvent_database_name='ecoinvent-3.10-cutoff', ecoinvent_version='3.10')

In [12]:
regio.spatialize_my_ecoinvent()

2025-09-17 14:25:35,053 - Regioinvent - INFO - Creating spatialized biosphere flows...
Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Title: Writing activities to SQLite3 database:
  Started: 09/17/2025 14:25:37
  Finished: 09/17/2025 14:25:46
  Total time elapsed: 00:00:08
  CPU %: 97.90
  Memory %: 1.20


2025-09-17 14:49:41,432 - Regioinvent - INFO - Extracting ecoinvent to wurst...


Getting activity data


100%|██████████| 23523/23523 [00:00<00:00, 32426.63it/s]


Adding exchange data to activities


100%|██████████| 743409/743409 [02:26<00:00, 5076.65it/s]


Filling out exchange data


100%|██████████| 23523/23523 [00:13<00:00, 1777.74it/s]
2025-09-17 14:52:48,208 - Regioinvent - INFO - Spatializing ecoinvent...
Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:15


Title: Writing activities to SQLite3 database:
  Started: 09/17/2025 14:52:58
  Finished: 09/17/2025 14:56:14
  Total time elapsed: 00:03:15
  CPU %: 97.60
  Memory %: 19.93


In [13]:
regio.import_fully_regionalized_impact_method(lcia_method='all')

2025-09-17 15:05:09,186 - Regioinvent - INFO - Importing all available fully regionalized lcia methods for ecoinvent3.10.


In [14]:
regio.regionalize_ecoinvent_with_trade(trade_database_path=r'C:\Users\mp_ma\OneDrive - polymtl\POST_DOC\Modules\Regioinvent\trade_data.db',
                                       regioinvent_database_name='Regioinvent',
                                       cutoff=0.99)

2025-09-17 15:06:48,277 - Regioinvent - INFO - Extracting and formatting trade data...
2025-09-17 15:07:22,537 - Regioinvent - INFO - Regionalizing main inputs of internationally-traded products of ecoinvent...
100%|██████████| 1982/1982 [47:36<00:00,  1.44s/it] 
2025-09-17 15:54:58,793 - Regioinvent - INFO - Regionalizing main inputs of non-internationally traded processes of ecoinvent...
100%|██████████| 67/67 [47:29<00:00, 42.52s/it]
2025-09-17 16:42:27,848 - Regioinvent - INFO - Creating consumption markets for internationally-traded products...
100%|██████████| 1982/1982 [1:47:29<00:00,  3.25s/it]  
2025-09-17 18:29:58,151 - Regioinvent - INFO - Link regioinvent processes to each other...
2025-09-17 18:30:22,440 - Regioinvent - INFO - Aggregate duplicates together...
2025-09-17 18:30:33,450 - Regioinvent - INFO - Regionalizing the elementary flows of the regioinvent database...
2025-09-17 18:30:38,974 - Regioinvent - INFO - Write regioinvent database to brightway...
Writing activi

Title: Writing activities to SQLite3 database:
  Started: 09/17/2025 18:30:46
  Finished: 09/17/2025 18:36:53
  Total time elapsed: 00:06:07
  CPU %: 99.60
  Memory %: 29.36


2025-09-17 18:46:35,484 - Regioinvent - INFO - Connecting ecoinvent to regioinvent processes...


# Custom LCIs

In [4]:
lci_brgm_path = r'not_in_github/Lai_et_al'
lci_files = [
    "Ag_LCIs_BW_Final_v2.xlsx",
    "Al_LCIs_BW_Final_v1.xlsx",
    "Au_LCIs_BW_Final_v1.xlsx",
    #"Cd_LCIs_BW_Final_v1.xlsx",
    "Cu_LCIs_BW_Final_v2.xlsx",
    "Fe_LCIs_BW_Final_v1.xlsx",
    #"Mo_LCIs_BW_Final_v1.xlsx",
    "Ni_LCIs_BW_Final_v1.xlsx",
    "Zn_LCIs_BW_Final_v2.xlsx",
    #"Te_LCIs_BW_Final_v2.xlsx",
]

In [5]:
for filename in lci_files:
    filepath = os.path.join(lci_brgm_path, filename)
    print(f"Importing: {filename}")

    importer = bw.ExcelImporter(filepath)
    importer.apply_strategies()

    importer.match_database("ecoinvent-3.10-cutoff", fields=("name", "reference product", "unit", "location"))
    importer.match_database("biosphere3", fields=("name", "unit", "categories"))

    unlinked = list(importer.unlinked)
    print(f"  → Unlinked exchanges: {len(unlinked)}")

    if not unlinked:
        importer.write_database()
        print(f"  → Database '{importer}' written.")
    else:
        print(f"  → Skipping '{filename}' due to unlinked exchanges.")


Importing: Ag_LCIs_BW_Final_v2.xlsx
Extracted 5 worksheets in 0.08 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 6.75 seconds
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


Applying strategy: link_iterable_by_fields
  → Unlinked exchanges: 0


0% [###########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/17/2025 13:55:55
  Finished: 09/17/2025 13:55:55
  Total time elapsed: 00:00:00
  CPU %: 151.20
  Memory %: 0.98
Created database: Silver
  → Database '<bw2io.importers.excel.ExcelImporter object at 0x0000024389A6BDC0>' written.
Importing: Al_LCIs_BW_Final_v1.xlsx
Extracted 14 worksheets in 0.17 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_ze

...

Title: Writing activities to SQLite3 database:
  Started: 09/17/2025 13:56:44
  Finished: 09/17/2025 13:56:44
  Total time elapsed: 00:00:00
  CPU %: 99.70
  Memory %: 1.12
Created database: Nickel
  → Database '<bw2io.importers.excel.ExcelImporter object at 0x0000024386766020>' written.
Importing: Zn_LCIs_BW_Final_v2.xlsx
Extracted 6 worksheets in 0.17 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zero

Writing activities to SQLite3 database:


Applying strategy: link_iterable_by_fields
  → Unlinked exchanges: 0


0% [######################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/17/2025 13:56:53
  Finished: 09/17/2025 13:56:53
  Total time elapsed: 00:00:00
  CPU %: 99.70
  Memory %: 1.14
Created database: Zinc
  → Database '<bw2io.importers.excel.ExcelImporter object at 0x00000243899EE8F0>' written.


# Export EI LCIs for selected activities

In [8]:
from lca_calculation_functions import get_inventory_dataset, run_lca, compute_midpoint_contributions

In [9]:
EI_DB = 'ecoinvent-3.10-cutoff'

In [12]:
INVENTORIES_EI = {
    "Copper concentrate": ("gold-silver mine operation and beneficiation", "copper concentrate, sulfide ore", 'CA-QC'),
    "Iron concentrate": ("iron ore mine operation and beneficiation", "iron ore concentrate", "CA-QC"),
    "Gold-silver ingot": ("gold-silver mine operation and beneficiation", "gold-silver, ingot", "CA-QC"),
    "Gold": ("gold-silver mine operation with refinery", "gold", "CA-QC"),
    "Silver": ("gold-silver mine operation with refinery", "silver", "CA-QC"),
    "Nickel concentrate": ("nickel mine operation and benefication to nickel concentrate, 16% Ni", "nickel concentrate, 16% Ni", "CA-QC"),
}

In [13]:
INVENTORIES_EI_ds = get_inventory_dataset(INVENTORIES_EI, database_names=[EI_DB])

In [14]:
INVENTORIES_EI_ds

{'Copper concentrate': 'gold-silver mine operation and beneficiation' (kilogram, CA-QC, None),
 'Iron concentrate': 'iron ore mine operation and beneficiation' (kilogram, CA-QC, None),
 'Gold-silver ingot': 'gold-silver mine operation and beneficiation' (kilogram, CA-QC, None),
 'Gold': 'gold-silver mine operation with refinery' (kilogram, CA-QC, None),
 'Silver': 'gold-silver mine operation with refinery' (kilogram, CA-QC, None),
 'Nickel concentrate': 'nickel mine operation and benefication to nickel concentrate, 16% Ni' (kilogram, CA-QC, None)}

In [15]:
def export_activity_exchanges(inventory_ds, output_folder="exports"):
    """
    Export technosphere and biosphere flows for each activity in inventory_ds.
    """
    import os
    os.makedirs(output_folder, exist_ok=True)

    all_rows = []  # to combine all results if needed

    for rm_name, act in inventory_ds.items():
        tech_exchanges = []
        bio_exchanges = []

        # --- Technosphere ---
        for exc in act.technosphere():
            tech_exchanges.append({
                "raw_material": rm_name,
                "activity_name": act["name"],
                "reference_product": act["reference product"],
                "location": act["location"],
                "exchange_type": "technosphere",
                "input_name": exc.input["name"],
                "input_database": exc.input["database"],
                "input_code": exc.input["code"],
                "amount": exc.amount,
                "unit": exc.input.get("unit", None),
                "comment": exc.get("comment", None)
            })

        # --- Biosphere ---
        for exc in act.biosphere():
            bio_exchanges.append({
                "raw_material": rm_name,
                "activity_name": act["name"],
                "reference_product": act["reference product"],
                "location": act["location"],
                "exchange_type": "biosphere",
                "input_name": exc.input["name"],
                "input_database": exc.input["database"],
                "input_code": exc.input["code"],
                "amount": exc.amount,
                "unit": exc.input.get("unit", None),
                "comment": exc.get("comment", None)
            })

        # --- Export individual CSVs ---
        pd.DataFrame(tech_exchanges).to_csv(
            f"{output_folder}/{rm_name}_technosphere.csv", index=False
        )
        pd.DataFrame(bio_exchanges).to_csv(
            f"{output_folder}/{rm_name}_biosphere.csv", index=False
        )

        all_rows.extend(tech_exchanges + bio_exchanges)

    # --- Optional: export all combined ---
    pd.DataFrame(all_rows).to_csv(f"{output_folder}/all_exchanges.csv", index=False)
    print(f"✅ Export complete: CSVs saved in '{output_folder}/'")

In [16]:
# 2️⃣ Export their technosphere & biosphere flows
export_activity_exchanges(INVENTORIES_EI_ds)

✅ Export complete: CSVs saved in 'exports/'
